In [2]:
import sqlite3, shutil

In [4]:
shutil.copy('openstudio_refrigeration_system.db', 'openstudio_refrigeration_system_wk.db')

'openstudio_refrigeration_system_wk.db'

##### Connect working database

In [7]:
conn=sqlite3.connect('openstudio_refrigeration_system_wk.db')
cursor = conn.cursor()

##### Determine user input/default system for Case and Walk-in